In [1]:
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as SPLIT
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('combined_cleaned.csv')



In [3]:
# modify birthyear-> 0-1 on a float scale
birthyears = df.birthyear.unique()
_min = min(birthyears)
_max = max(birthyears)

def normalize_birthyear(year):
    return (year-_min)/(_max-_min)

birthyear_labels = df.birthyear.apply(normalize_birthyear)
birthyear_labels = birthyear_labels.values
birthyear_labels

array([0.64705882, 0.30882353, 0.72058824, ..., 0.45588235, 0.73529412,
       0.58823529])

In [4]:
labels_to_onehot = ['fame', 'gender', 'occupation']

from sklearn.preprocessing import LabelBinarizer

labels = {}

for label in labels_to_onehot:
    unique_classes = len(df[label].unique())
    print('{} unique classes in {}'.format(unique_classes, label))
    labels_nd = df[label].values
    labels[label] = labels_nd
    #encoder = LabelBinarizer()
    #labels[label] = encoder.fit_transform(labels_nd)
    labels[label] = pd.get_dummies(labels_nd)

2 unique classes in fame
3 unique classes in gender
8 unique classes in occupation


In [5]:
fame_labels = labels['fame']
gender_labels = labels['gender']
occ_labels = labels['occupation']

In [6]:
#fame_labels = to_categorical(fame_labels)
#gender_labels = to_categorical(gender_labels)
#occ_labels = to_categorical(occ_labels)

print(fame_labels.shape)
print(gender_labels.shape)
print(occ_labels.shape)

(6996, 2)
(6996, 3)
(6996, 8)


In [7]:
from keras.preprocessing.text import Tokenizer
vocab_size = 15000
 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df.text)
 
x_train = tokenizer.texts_to_matrix(df.text, mode='tfidf')

In [8]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizer.pickle', 'rb') as handle:
#    tokenizer = pickle.load(handle)

In [9]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

2

In [10]:
shape = (vocab_size,)
input_layer = Input(shape)
# before splitting categories
branch_layer = Dense(units=128, activation='relu')(input_layer)
# birthyear
_ = Dense(units=128, activation='relu')(branch_layer)
birthyear_out = Dense(units=1, activation='sigmoid', name='birthyear_out')(_)

# fame
_ = Dense(units=128, activation='relu')(branch_layer)
fame_out = Dense(units=fame_labels.shape[1], activation='softmax', name='fame_out')(_)

# gender
_ = Dense(units=128, activation='relu')(branch_layer)
gender_out = Dense(units=gender_labels.shape[1], activation='softmax', name='gender_out')(_)

# occupation
_ = Dense(units=128, activation='relu')(branch_layer)
occ_out = Dense(units=occ_labels.shape[1], activation='softmax', name='occ_out')(_)

model = Model(input=input_layer, outputs=[birthyear_out, fame_out, gender_out, occ_out])
#model = Model(input=input_layer, outputs=fame_out)
loss_fn = 'categorical_crossentropy'

model.compile(optimizer='adam',
              loss={'birthyear_out': 'mse', 'fame_out': loss_fn, 'gender_out': loss_fn, 'occ_out': loss_fn},
              metrics={'birthyear_out': 'mae', 'fame_out': 'accuracy', 'gender_out': 'accuracy', 'occ_out': 'accuracy'}
             )

#model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\Tollef\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15000)        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          1920128     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

In [17]:
#board = keras.callbacks.TensorBoard(log_dir='./tensorboard', histogram_freq=0, write_graph=True, write_images=True)
board = keras.callbacks.TensorBoard(log_dir='./tensorboard', histogram_freq=0)

In [20]:
y_train = [birthyear_labels, fame_labels, gender_labels, occ_labels]
model.fit(x_train, y_train, epochs=2000, batch_size=32, validation_split=0.1, callbacks=[board])


Train on 6296 samples, validate on 700 samples
Epoch 1/2000
6296/6296 [==============================] - 3s 436us/step - loss: 2.7328 - birthyear_out_loss: 0.1668 - fame_out_loss: 0.3204 - gender_out_loss: 0.5530 - occ_out_loss: 1.6925 - birthyear_out_mean_absolute_error: 0.3743 - fame_out_acc: 0.9787 - gender_out_acc: 0.9657 - occ_out_acc: 0.8945 - val_loss: 2.6191 - val_birthyear_out_loss: 0.1595 - val_fame_out_loss: 0.0154 - val_gender_out_loss: 0.6686 - val_occ_out_loss: 1.7757 - val_birthyear_out_mean_absolute_error: 0.3548 - val_fame_out_acc: 0.9986 - val_gender_out_acc: 0.9571 - val_occ_out_acc: 0.8829
Epoch 2/2000
6296/6296 [==============================] - 3s 436us/step - loss: 2.7088 - birthyear_out_loss: 0.1669 - fame_out_loss: 0.3072 - gender_out_loss: 0.5086 - occ_out_loss: 1.7261 - birthyear_out_mean_absolute_error: 0.3745 - fame_out_acc: 0.9800 - gender_out_acc: 0.9682 - occ_out_acc: 0.8922 - val_loss: 2.1727 - val_birthyear_out_loss: 0.1594 - val_fame_out_loss: 0.0111 

6296/6296 [==============================] - 3s 422us/step - loss: 2.4594 - birthyear_out_loss: 0.1639 - fame_out_loss: 0.2847 - gender_out_loss: 0.4107 - occ_out_loss: 1.6001 - birthyear_out_mean_absolute_error: 0.3714 - fame_out_acc: 0.9821 - gender_out_acc: 0.9743 - occ_out_acc: 0.9004 - val_loss: 2.2400 - val_birthyear_out_loss: 0.1675 - val_fame_out_loss: 0.0541 - val_gender_out_loss: 0.2085 - val_occ_out_loss: 1.8098 - val_birthyear_out_mean_absolute_error: 0.3627 - val_fame_out_acc: 0.9943 - val_gender_out_acc: 0.9857 - val_occ_out_acc: 0.8829
Epoch 16/2000
6296/6296 [==============================] - 3s 425us/step - loss: 2.4456 - birthyear_out_loss: 0.1661 - fame_out_loss: 0.2859 - gender_out_loss: 0.3359 - occ_out_loss: 1.6578 - birthyear_out_mean_absolute_error: 0.3734 - fame_out_acc: 0.9819 - gender_out_acc: 0.9790 - occ_out_acc: 0.8969 - val_loss: 2.1713 - val_birthyear_out_loss: 0.1601 - val_fame_out_loss: 0.0050 - val_gender_out_loss: 0.2763 - val_occ_out_loss: 1.7299 - 

6296/6296 [==============================] - 3s 418us/step - loss: 2.5994 - birthyear_out_loss: 0.1617 - fame_out_loss: 0.2814 - gender_out_loss: 0.3338 - occ_out_loss: 1.8224 - birthyear_out_mean_absolute_error: 0.3691 - fame_out_acc: 0.9821 - gender_out_acc: 0.9790 - occ_out_acc: 0.8858 - val_loss: 2.3192 - val_birthyear_out_loss: 0.1629 - val_fame_out_loss: 0.0566 - val_gender_out_loss: 0.1612 - val_occ_out_loss: 1.9386 - val_birthyear_out_mean_absolute_error: 0.3589 - val_fame_out_acc: 0.9943 - val_gender_out_acc: 0.9900 - val_occ_out_acc: 0.8786
Epoch 30/2000
6296/6296 [==============================] - 3s 418us/step - loss: 2.4453 - birthyear_out_loss: 0.1640 - fame_out_loss: 0.2720 - gender_out_loss: 0.4054 - occ_out_loss: 1.6039 - birthyear_out_mean_absolute_error: 0.3715 - fame_out_acc: 0.9830 - gender_out_acc: 0.9746 - occ_out_acc: 0.9003 - val_loss: 2.4341 - val_birthyear_out_loss: 0.1606 - val_fame_out_loss: 0.0479 - val_gender_out_loss: 0.2634 - val_occ_out_loss: 1.9621 - 

6296/6296 [==============================] - 3s 417us/step - loss: 2.3784 - birthyear_out_loss: 0.1671 - fame_out_loss: 0.2837 - gender_out_loss: 0.3251 - occ_out_loss: 1.6026 - birthyear_out_mean_absolute_error: 0.3746 - fame_out_acc: 0.9821 - gender_out_acc: 0.9798 - occ_out_acc: 0.9003 - val_loss: 2.2170 - val_birthyear_out_loss: 0.1635 - val_fame_out_loss: 8.7929e-05 - val_gender_out_loss: 0.3454 - val_occ_out_loss: 1.7080 - val_birthyear_out_mean_absolute_error: 0.3593 - val_fame_out_acc: 1.0000 - val_gender_out_acc: 0.9786 - val_occ_out_acc: 0.8929
Epoch 44/2000
6296/6296 [==============================] - 3s 421us/step - loss: 2.3950 - birthyear_out_loss: 0.1679 - fame_out_loss: 0.2820 - gender_out_loss: 0.3767 - occ_out_loss: 1.5684 - birthyear_out_mean_absolute_error: 0.3755 - fame_out_acc: 0.9824 - gender_out_acc: 0.9765 - occ_out_acc: 0.9026 - val_loss: 2.1896 - val_birthyear_out_loss: 0.1747 - val_fame_out_loss: 0.0231 - val_gender_out_loss: 0.2763 - val_occ_out_loss: 1.715

6296/6296 [==============================] - 3s 425us/step - loss: 2.4058 - birthyear_out_loss: 0.1630 - fame_out_loss: 0.3067 - gender_out_loss: 0.3600 - occ_out_loss: 1.5760 - birthyear_out_mean_absolute_error: 0.3706 - fame_out_acc: 0.9795 - gender_out_acc: 0.9773 - occ_out_acc: 0.9018 - val_loss: 2.6101 - val_birthyear_out_loss: 0.1609 - val_fame_out_loss: 0.0248 - val_gender_out_loss: 0.5066 - val_occ_out_loss: 1.9179 - val_birthyear_out_mean_absolute_error: 0.3567 - val_fame_out_acc: 0.9971 - val_gender_out_acc: 0.9686 - val_occ_out_acc: 0.8800
Epoch 58/2000
6296/6296 [==============================] - 3s 419us/step - loss: 2.3739 - birthyear_out_loss: 0.1625 - fame_out_loss: 0.2737 - gender_out_loss: 0.3556 - occ_out_loss: 1.5821 - birthyear_out_mean_absolute_error: 0.3700 - fame_out_acc: 0.9827 - gender_out_acc: 0.9778 - occ_out_acc: 0.9018 - val_loss: 2.2744 - val_birthyear_out_loss: 0.1641 - val_fame_out_loss: 0.0430 - val_gender_out_loss: 0.2763 - val_occ_out_loss: 1.7910 - 

...

6296/6296 [==============================] - 3s 428us/step - loss: 2.1771 - birthyear_out_loss: 0.1593 - fame_out_loss: 0.2817 - gender_out_loss: 0.2355 - occ_out_loss: 1.5007 - birthyear_out_mean_absolute_error: 0.3662 - fame_out_acc: 0.9822 - gender_out_acc: 0.9854 - occ_out_acc: 0.9068 - val_loss: 3.7962 - val_birthyear_out_loss: 0.1614 - val_fame_out_loss: 0.0691 - val_gender_out_loss: 0.6471 - val_occ_out_loss: 2.9186 - val_birthyear_out_mean_absolute_error: 0.3568 - val_fame_out_acc: 0.9957 - val_gender_out_acc: 0.9586 - val_occ_out_acc: 0.8186
Epoch 729/2000
6296/6296 [==============================] - 3s 428us/step - loss: 2.2783 - birthyear_out_loss: 0.1575 - fame_out_loss: 0.3163 - gender_out_loss: 0.3075 - occ_out_loss: 1.4969 - birthyear_out_mean_absolute_error: 0.3644 - fame_out_acc: 0.9797 - gender_out_acc: 0.9808 - occ_out_acc: 0.9071 - val_loss: 5.0430 - val_birthyear_out_loss: 0.1607 - val_fame_out_loss: 0.2085 - val_gender_out_loss: 2.0723 - val_occ_out_loss: 2.6014 -

6296/6296 [==============================] - 3s 423us/step - loss: 2.1013 - birthyear_out_loss: 0.1777 - fame_out_loss: 0.2545 - gender_out_loss: 0.2176 - occ_out_loss: 1.4516 - birthyear_out_mean_absolute_error: 0.3845 - fame_out_acc: 0.9841 - gender_out_acc: 0.9865 - occ_out_acc: 0.9099 - val_loss: 3.4602 - val_birthyear_out_loss: 0.1882 - val_fame_out_loss: 0.1159 - val_gender_out_loss: 0.5070 - val_occ_out_loss: 2.6491 - val_birthyear_out_mean_absolute_error: 0.3839 - val_fame_out_acc: 0.9929 - val_gender_out_acc: 0.9686 - val_occ_out_acc: 0.8343
Epoch 743/2000
6296/6296 [==============================] - 3s 424us/step - loss: 2.0751 - birthyear_out_loss: 0.1743 - fame_out_loss: 0.2261 - gender_out_loss: 0.2099 - occ_out_loss: 1.4648 - birthyear_out_mean_absolute_error: 0.3812 - fame_out_acc: 0.9859 - gender_out_acc: 0.9870 - occ_out_acc: 0.9090 - val_loss: 3.5541 - val_birthyear_out_loss: 0.1809 - val_fame_out_loss: 0.1490 - val_gender_out_loss: 0.5084 - val_occ_out_loss: 2.7158 -

6296/6296 [==============================] - 3s 424us/step - loss: 2.6723 - birthyear_out_loss: 0.1629 - fame_out_loss: 0.2236 - gender_out_loss: 0.7090 - occ_out_loss: 1.5768 - birthyear_out_mean_absolute_error: 0.3698 - fame_out_acc: 0.9857 - gender_out_acc: 0.9558 - occ_out_acc: 0.9020 - val_loss: 3.9634 - val_birthyear_out_loss: 0.1643 - val_fame_out_loss: 0.1743 - val_gender_out_loss: 1.1256 - val_occ_out_loss: 2.4991 - val_birthyear_out_mean_absolute_error: 0.3598 - val_fame_out_acc: 0.9871 - val_gender_out_acc: 0.9300 - val_occ_out_acc: 0.8414
Epoch 757/2000
6296/6296 [==============================] - 3s 432us/step - loss: 2.0991 - birthyear_out_loss: 0.1573 - fame_out_loss: 0.2125 - gender_out_loss: 0.2381 - occ_out_loss: 1.4912 - birthyear_out_mean_absolute_error: 0.3642 - fame_out_acc: 0.9868 - gender_out_acc: 0.9852 - occ_out_acc: 0.9074 - val_loss: 3.4591 - val_birthyear_out_loss: 0.1651 - val_fame_out_loss: 0.1347 - val_gender_out_loss: 0.7011 - val_occ_out_loss: 2.4582 -

6296/6296 [==============================] - 3s 421us/step - loss: 2.0823 - birthyear_out_loss: 0.1608 - fame_out_loss: 0.2048 - gender_out_loss: 0.3111 - occ_out_loss: 1.4055 - birthyear_out_mean_absolute_error: 0.3678 - fame_out_acc: 0.9873 - gender_out_acc: 0.9806 - occ_out_acc: 0.9128 - val_loss: 3.7941 - val_birthyear_out_loss: 0.1667 - val_fame_out_loss: 0.3954 - val_gender_out_loss: 0.7138 - val_occ_out_loss: 2.5182 - val_birthyear_out_mean_absolute_error: 0.3621 - val_fame_out_acc: 0.9743 - val_gender_out_acc: 0.9557 - val_occ_out_acc: 0.8429
Epoch 771/2000
6296/6296 [==============================] - 3s 426us/step - loss: 2.0294 - birthyear_out_loss: 0.1637 - fame_out_loss: 0.1991 - gender_out_loss: 0.2586 - occ_out_loss: 1.4080 - birthyear_out_mean_absolute_error: 0.3706 - fame_out_acc: 0.9876 - gender_out_acc: 0.9840 - occ_out_acc: 0.9126 - val_loss: 3.6429 - val_birthyear_out_loss: 0.1686 - val_fame_out_loss: 0.1473 - val_gender_out_loss: 0.9210 - val_occ_out_loss: 2.4060 -

KeyboardInterrupt: 